In [1]:
import numpy as np
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch
from sklearn.model_selection import KFold
import wandb
import os

random.seed(27)
np.random.seed(27)
plt.style.use('ggplot')

In [2]:
from NN.denseLayer import *
from NN.loss import *
from NN.optimizer import *
from NN.activations import *

## Dataset

FashionMNIST.

In [3]:
num_classes = 10

In [4]:
def load_fashion_mnist(flatten=True):
    transform = transforms.ToTensor()

    train_ds = datasets.FashionMNIST(
        root="data",
        train=True,
        download=True,
        transform=transform
    )

    test_ds = datasets.FashionMNIST(
        root="data",
        train=False,
        download=True,
        transform=transform
    )

    # Convert to numpy
    X_train = train_ds.data.numpy().astype(np.float32)
    y_train = train_ds.targets.numpy()
    X_test = test_ds.data.numpy().astype(np.float32)
    y_test = test_ds.targets.numpy()

    # Normalize
    X_train /= 255.0
    X_test /= 255.0

    # Flatten 
    if flatten:
        X_train = X_train.reshape(len(X_train), -1)
        X_test = X_test.reshape(len(X_test), -1)
    
    return X_train, y_train, X_test, y_test

In [5]:
def one_hot(y, num_classes):
    Y = np.zeros((len(y), num_classes))
    Y[np.arange(len(y)), y] = 1
    return Y

def accuracy(probs, Y_true):
    preds = np.argmax(probs, axis=0)
    labels = np.argmax(Y_true, axis=0)
    return np.mean(preds == labels)

In [6]:
X_train, y_train, X_test, y_test = load_fashion_mnist(flatten=True) 

100%|██████████| 26.4M/26.4M [00:27<00:00, 958kB/s] 
100%|██████████| 29.5k/29.5k [00:00<00:00, 1.31MB/s]
100%|██████████| 4.42M/4.42M [00:03<00:00, 1.19MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 29.6MB/s]


In [7]:
Y_train = one_hot(y_train, num_classes).T  # (10, 60000)
Y_test  = one_hot(y_test, num_classes).T   # (10, 10000)

print("X_train:", X_train.shape, "Y_train:", Y_train.shape)
print("X_test: ", X_test.shape,  "Y_test: ",  Y_test.shape)

X_train: (60000, 784) Y_train: (10, 60000)
X_test:  (10000, 784) Y_test:  (10, 10000)


## Sweep

Helper functions:

In [8]:
def get_activation_instance(name):
    if name == "relu":
        return ReLU()
    elif name == "tanh":
        return Tanh()
    elif name == "sigmoid":
        return Sigmoid()

def get_optimizer(name, lr):
    if name == "sgd":
        return SGD(learning_rate=lr)
    elif name == "adam":
        return Adam(learning_rate=lr)

Sweep configuration:

In [9]:
random_sweep = {
    "method": "random",
    "metric": {
        "name": "val_accuracy",
        "goal": "maximize"
    },
    "parameters": {
        "learning_rate": {"distribution": "log_uniform_values", "min": 0.0001, "max": 0.01},
        "batch_size": {"values": [32, 64, 128]},
        "epochs": {"value": 30},
        "num_hidden_layers": {"distribution": "int_uniform", "min": 1, "max": 5},
        "n_hidden_units": {"distribution": "int_uniform", "min": 32, "max": 128},
        "l2_coeff": {"distribution": "uniform", "min": 0.0, "max": 0.001},
        "optimizer": {"values": ["sgd", "adam"]},
        "activation": {"values": ["relu", "tanh", "sigmoid"]},
        "num_classes": {"value": num_classes},
    },
}

In [10]:
bayes_sweep = {
    "method": "bayes",
    "metric": {"name": "mean_cv_accuracy", "goal": "maximize"},
    "parameters": {
        "learning_rate": {"distribution": "log_uniform_values", "min": 0.0001, "max": 0.01},
        "batch_size": {"values": [32, 64, 128]},
        "epochs": {"value": 30},
        "num_hidden_layers": {"distribution": "int_uniform", "min": 1, "max": 5},
        "n_hidden_units": {"distribution": "int_uniform", "min": 32, "max": 128},
        "l2_coeff": {"distribution": "uniform", "min": 0.0, "max": 0.001},
        "optimizer": {"values": ["sgd", "adam"]},
        "activation": {"values": ["relu", "tanh", "sigmoid"]},
        "num_classes": {"value": num_classes},
    },
}

In [11]:
random_sweep_id = wandb.sweep(random_sweep, project="numpy-nn-random-search-cv")

Create sweep with ID: icx3j71h
Sweep URL: https://wandb.ai/xanderbaatz-danmarks-tekniske-universitet-dtu/numpy-nn-random-search-cv/sweeps/icx3j71h


In [12]:
bayesian_sweep_id = wandb.sweep(sweep=bayes_sweep, project="numpy-nn-bayes-cv")

Create sweep with ID: i0egd2xn
Sweep URL: https://wandb.ai/xanderbaatz-danmarks-tekniske-universitet-dtu/numpy-nn-bayes-cv/sweeps/i0egd2xn


## Train procedure

In [13]:
#os.environ['WANDB_API_KEY'] = ...

wandb.login()

wandb: Currently logged in as: xanderbaatz (xanderbaatz-danmarks-tekniske-universitet-dtu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [14]:
def stack_layers(input_dim, cfg):
    layers = []
    prev = input_dim

    # For all the layers (0, l-1)
    for _ in range(int(cfg['num_hidden_layers'])):
        act = get_activation_instance(cfg['activation'])
        layers.append(DenseLayer(prev, int(cfg['n_hidden_units']), activation=act,
                                 initializer=None, l2_coeff=float(cfg.get('l2_coeff', 0.0)))) #We set initialiser to none, since we have activation functions as hyperparatmerters, and denselayer takes care of what initialiser to use
        prev = int(cfg['n_hidden_units'])

    # Final layer
    layers.append(DenseLayer(prev, int(cfg['num_classes']), activation=Softmax(),
                             initializer=None))
    return layers

#Get the accuracy metric
def accuracy_from_probs(A, Y_true_onehot):
    preds = A.argmax(axis=0)
    truths = Y_true_onehot.argmax(axis=0)
    return float((preds == truths).mean())

def train_on_split(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, cfg):
    layers = stack_layers(X_train_fold.shape[1], cfg)
    optimizer = get_optimizer(cfg['optimizer'], float(cfg['learning_rate']))
    loss_fn = CrossEntropyLoss()

    m = X_train_fold.shape[0]
    epochs = int(cfg["epochs"])
    batch_size = int(cfg["batch_size"])

    epoch_loss = 0.0
    num_batches = 0

    #For each epoch, we perform batching, do the usual forward, backwards measure
    #again this is summarised in the FFN-explainer notebook.
    for epoch in range(epochs):
        perm = np.random.permutation(m)

        Xs = X_train_fold[perm].T    
        Ys = Y_train_fold[:, perm]    

        for i in range(0, m, batch_size):
            Xb = Xs[:, i:i+batch_size]
            Yb = Ys[:, i:i+batch_size]
            
            A = Xb
            for layer in layers:
                A = layer.forward(A)

            loss = loss_fn.forward(A, Yb)

            epoch_loss += loss
            num_batches += 1

            dA = loss_fn.backward(A, Yb)

            for layer in reversed(layers):
                dA, dW, db = layer.backward(dA)
                optimizer.update(layer, dW, db)
        
    train_loss = epoch_loss / num_batches

    A_train = Xs
    for layer in layers:
        A_train = layer.forward(A_train)
    train_acc = accuracy_from_probs(A_train, Ys)

    A_val = X_val_fold.T
    for layer in layers:
        A_val = layer.forward(A_val)

    val_loss = loss_fn.forward(A_val, Y_val_fold)
    val_acc = accuracy_from_probs(A_val, Y_val_fold)

    wandb.log({
            "train_loss": float(train_loss),
            "train_accuracy": float(train_acc),
            "val_loss": float(val_loss),
            "val_accuracy": float(val_acc)
        })

    return val_acc, val_loss

def train_run():
    with wandb.init() as run:
        cfg = dict(run.config)
        run.log(dict(cfg))

        kf = KFold(n_splits=3, shuffle=True, random_state=27)
        fold_accuracies = []
        fold_val_losses = []

        #For N k-folds, train the split, and return the metrics
        for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X_train)):

            X_train_fold = X_train[train_idx]
            Y_train_fold = Y_train[:, train_idx]

            X_val_fold = X_train[val_idx]
            Y_val_fold = Y_train[:, val_idx]

            val_acc, val_loss = train_on_split(
                X_train_fold, Y_train_fold,
                X_val_fold, Y_val_fold,
                cfg
            )

            fold_accuracies.append(val_acc)
            fold_val_losses.append(val_loss)
            run.log({f"fold_{fold_idx+1}_accuracy": float(val_acc)})
            run.log({f"fold_{fold_idx+1}_loss": float(val_loss)})

        #Now we find the generlization error
        mean_cv_acc = float(np.mean(fold_accuracies))
        acc_cv_std = float(np.std(fold_accuracies))
        mean_cv_loss = float(np.mean(fold_val_losses))
        loss_cv_std = float(np.std(fold_val_losses))
        run.log({"mean_cv_accuracy": mean_cv_acc})
        run.log({"accuracy_cv_std": acc_cv_std})
        run.log({"mean_cv_loss": mean_cv_loss})
        run.log({"loss_cv_std": loss_cv_std})

        print(f"Run {run.id} \nCV Accuracies={fold_accuracies} \nMean={mean_cv_acc:.4f}")
        print(f"CV Losses={fold_val_losses} \nMean={mean_cv_loss:.4f}")

        wandb.finish()

In [16]:
# Bayesian
wandb.agent(bayesian_sweep_id, function=train_run, count=30)

wandb: Agent Starting Run: tf3krxy9 with config:
wandb: 	activation: tanh
wandb: 	batch_size: 128
wandb: 	epochs: 30
wandb: 	l2_coeff: 7.562229622513762e-05
wandb: 	learning_rate: 0.00010294482719454676
wandb: 	n_hidden_units: 103
wandb: 	num_classes: 10
wandb: 	num_hidden_layers: 5
wandb: 	optimizer: adam


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Run tf3krxy9 
CV Accuracies=[0.88265, 0.8843, 0.88345] 
Mean=0.8835
CV Losses=[np.float64(0.33927505945984454), np.float64(0.33113124747259204), np.float64(0.331664947895519)] 
Mean=0.3340


accuracy_cv_std,▁
batch_size,▁
epochs,▁
fold_1_accuracy,▁
fold_1_loss,▁
fold_2_accuracy,▁
fold_2_loss,▁
fold_3_accuracy,▁
fold_3_loss,▁
l2_coeff,▁
+11,...


wandb: Agent Starting Run: mfbl721x with config:
wandb: 	activation: tanh
wandb: 	batch_size: 128
wandb: 	epochs: 30
wandb: 	l2_coeff: 0.0001762343345770151
wandb: 	learning_rate: 0.0003225147384664596
wandb: 	n_hidden_units: 121
wandb: 	num_classes: 10
wandb: 	num_hidden_layers: 2
wandb: 	optimizer: adam


...

accuracy_cv_std,▁
batch_size,▁
epochs,▁
fold_1_accuracy,▁
fold_1_loss,▁
fold_2_accuracy,▁
fold_2_loss,▁
fold_3_accuracy,▁
fold_3_loss,▁
l2_coeff,▁
+11,...


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: r44e49ss with config:
wandb: 	activation: relu
wandb: 	batch_size: 32
wandb: 	epochs: 30
wandb: 	l2_coeff: 0.00022752476835996192
wandb: 	learning_rate: 0.00606526828608985
wandb: 	n_hidden_units: 125
wandb: 	num_classes: 10
wandb: 	num_hidden_layers: 1
wandb: 	optimizer: adam


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Run r44e49ss 
CV Accuracies=[0.8533, 0.85755, 0.85005] 
Mean=0.8536
CV Losses=[np.float64(0.41111387882493483), np.float64(0.39229848819569163), np.float64(0.42896995698731233)] 
Mean=0.4108


accuracy_cv_std,▁
batch_size,▁
epochs,▁
fold_1_accuracy,▁
fold_1_loss,▁
fold_2_accuracy,▁
fold_2_loss,▁
fold_3_accuracy,▁
fold_3_loss,▁
l2_coeff,▁
+11,...
